In [9]:
# create tensor files
import os, sys
import gensim
from gensim.models import KeyedVectors

eebo_models=list()
input_data = ["../models/eebo-henry_VII.w2v",
              "../models/eebo-henry_VIII.w2v",
              "../models/eebo-edward_VI.w2v",
              "../models/eebo-mary_I.w2v",
              "../models/eebo-elizabeth_I.w2v",
              "../models/eebo-james_I.w2v",
              "../models/eebo-charles_I.w2v",
              "../models/eebo-charles_II.w2v",
              "../models/eebo-oliver_cromwell.w2v"]

for fp in input_data:
    model_name = os.path.basename(fp).split(".")[0]
    print("starting: {0}".format(model_name))
    eebo_models.append([model_name,KeyedVectors.load(fp,mmap='r')])

starting: eebo-henry_VII
starting: eebo-henry_VIII
starting: eebo-edward_VI
starting: eebo-mary_I
starting: eebo-elizabeth_I
starting: eebo-james_I
starting: eebo-charles_I
starting: eebo-charles_II
starting: eebo-oliver_cromwell


In [13]:
eebo_models[0][1]["faith"]

array([-1.2982992 ,  1.329542  , -1.6990196 ,  1.9866759 ,  0.34680083,
        0.6122618 ,  0.87340933,  0.03618077, -0.21907102,  0.9482734 ,
        0.12522952,  0.79288983,  0.8030496 ,  0.39939988,  1.1727644 ,
        0.95504117,  0.7252918 , -1.1681837 , -0.06358011,  0.16742656,
       -1.3222063 ,  0.87567097,  0.24866885, -0.6291752 ,  0.9448509 ,
        0.5238228 , -0.4567353 ,  0.2352058 ,  0.7236827 , -0.30481482,
       -0.7457204 , -0.1659573 ,  0.6963823 , -0.12151036, -1.073753  ,
        0.91585374, -0.5835327 ,  0.58553445, -0.73697746, -0.17457974,
        0.11087843, -1.7193842 , -0.66785586,  0.45017406,  0.02621414,
        0.63623005, -0.9002279 , -0.23966171,  0.36332783,  0.7154924 ,
       -0.52566224,  0.05294343,  0.5709484 ,  0.81315124, -0.7757787 ,
       -0.83805823,  0.12496756, -0.3175425 ,  0.90506303,  0.37474096,
        0.5144583 ,  0.5758533 , -0.10591204, -1.0417677 , -0.29347888,
        1.6737188 , -0.22962125,  0.7697354 ,  0.15588452, -0.22

In [12]:
eebo_models[0][1].most_similar("faith",topn=100)

[('fayth', 0.6269285082817078),
 ('faythe', 0.6234911680221558),
 ('feythe', 0.6082746386528015),
 ('feyth', 0.5719614028930664),
 ('enformed', 0.5204412937164307),
 ('prechyng', 0.5196342468261719),
 ('deuyne', 0.5107433795928955),
 ('faithe', 0.509698212146759),
 ('feith', 0.508242666721344),
 ('scy', 0.5079211592674255),
 ('letterly', 0.4996170699596405),
 ('ence', 0.49085524678230286),
 ('iustice', 0.4879233241081238),
 ('byleue', 0.48755350708961487),
 ('heretykes', 0.4870219826698303),
 ('scisme', 0.48688942193984985),
 ('scryptures', 0.4859585165977478),
 ('consideryng', 0.48498526215553284),
 ('ticles', 0.4749033749103546),
 ('exhortyng', 0.4734031558036804),
 ('merites', 0.47194021940231323),
 ('resseyue', 0.4713841378688812),
 ('obe', 0.4708738923072815),
 ('memory', 0.4704833924770355),
 ('constaunte', 0.46838369965553284),
 ('grettist', 0.4680805504322052),
 ('meruelê\x9d°', 0.4680333137512207),
 ('conuersacion', 0.4657440483570099),
 ('defendyng', 0.4639008045196533),
 ('c

In [14]:
outfilejson = "../tensors/tensors_all.json"

json_data = dict()
json_data['embeddings'] = list()

# limit model
key_terms = ["religion","church","self","fate","reformation",
             "elect","protestant","chosen","faith",
             "predestination"]

for m in eebo_models:
    print(m[0])
    
    tensor_filename = "../tensors/" + m[0] 
    outfiletsv = tensor_filename + '_tensor.tsv'
    outfiletsvmeta = tensor_filename + '_metadata.tsv'
    
    vectors=list()
    vocab=list()
    for t in key_terms:
        if t in m[1].vocab:
            nn = [x[0] for x in m[1].most_similar(t,topn=100)]
            for n in nn:
                vocab.append(n)
    vocab = set(vocab)
    
    with open(outfiletsv, 'wt') as file_vector:
        with open(outfiletsvmeta, 'wt',encoding='utf-8') as file_metadata:
            for word in vocab:
                print(word)
                #file_metadata.write(word) + '\n'
                vector_row = '\t'.join(str(x) for x in m[1][word])
                file_vector.write(vector_row + '\n')
                
    shape = [1000,50]
    json_data['embeddings'].append({
        'tensorName': str(m[0]),
        'tensorPath': 'https://raw.githubusercontent.com/catherinedparnell/Cultural-Analytics-IS/master/tensors/concepts-' + str(model_name) + '_tensor.tsv',
        'metadataPath': 'https://raw.githubusercontent.com/catherinedparnell/Cultural-Analytics-IS/master/tensors/concepts-' + str(model_name) + '_metadata.tsv',
        'tensorShape' : shape
    })
                

eebo-henry_VII
gooste
lawes
relegyous
kinrede
auctorities
translated
colleges
sygysmundus
holynesse
seruice
aym
appostles
sacramentes
martir
acqui
officers
frowardneå¿å¿e
vnwarly
representacion
pture
pleå¿aunce
soo
clergy
aoth
ca.xix
ghost
martres
fraunches
chauncheler
geuen
êsonales
sheweth
shaftesburye
wysdom
baptised
cuå¿tum
falå¿ly
preceptes
cretes
å¿elf
å¿ignificaå¿ti
lefulle
chastise
iue
special
goost
conå¿ecrate
lombardys
ptholomeê°
furious
beryngarius
benefyce
moeuable
sacram
odorus
vnhylled
candulus
manilius
desire
successyon
chyrch
calixt
wrappid
arnulphus
conå¿tantin
synnes
confermed
gost
crystys
å¿ext
obserued
lenyng
oold
teneme
choå¿in
vnable
dolobela
grauntyd
rightful
confirmacion
publike
cryå¿tente
clement
archbisshop
anentis
seculer
hymself
doggê­
articles
creå¿me
wedlok
wisdom
scrypture
moost
saue
strength
innocencius
å¿yngyuge
seknesses
almo
lens
gooå¿te
osteryk
otto
religyon
maydes
cho
gooå¿t
vnableth
creples
adamonde
marciall
chastyse
pease
pollicene
toryte
deuy

In [16]:
import json
with open(outfilejson, 'wt') as fp:
    json.dump(json_data, fp)

In [6]:
vocab

{"'d",
 "'re",
 "'ve",
 '1.4.5.',
 '374',
 '41.2',
 '432',
 '628',
 '649',
 '848.',
 '882',
 '``',
 'abolition',
 'aboliå¿hing',
 'abouesayd',
 'abr',
 'abraham',
 'abrahams',
 'abrahen',
 'absolved',
 'abuses',
 'accomplishment',
 'accused',
 'actual',
 'actuall',
 'actually',
 'administration',
 'adoption',
 'affiance',
 'agen',
 'almost',
 'also',
 'ambition',
 'amynt',
 'anabaptists',
 'angels',
 'antecedent',
 'antichrist',
 'antichristian',
 'antichriå¿tian',
 'aphorism',
 'apo',
 'apostasie',
 'apostles',
 'apostolical',
 'apoå¿tacy',
 'apoå¿tolicum',
 'appointed',
 'appointment',
 'apprehension',
 'archbishops',
 'argalia',
 'argum',
 'armie-saints',
 'arrians',
 'artamenes',
 'assembled',
 'assemblies',
 'assent',
 'asserted',
 'asserting',
 'assertion',
 'asserts',
 'assigned',
 'assistant',
 'assurance',
 'augustan',
 'azymorum',
 'aå¿å¿urance',
 'babylonish',
 'banished',
 'baptized',
 'baptiå¿m',
 'bation',
 'beauties',
 'because',
 'begetting',
 'begotten',
 'beleefe',
 '